# apply mask on image

In [ ]:

def mask_on_image(im , mask, masking_type):
    new_pic=[]
    new_im=np.transpose(im, (2, 0, 1))
    if masking_type=='normal/':
        for i in new_im:
            new_pic.append(i*(~mask))
    else:
        for i in new_im:
            new_pic.append(i*(mask))
    new_im=np.transpose(new_pic, (1,2,0))
    return new_im

In [ ]:
def show_redacted_image(segmentwise_attributions,prediction_id,redaction1_percentile):
    masking_type='normal/'
    mask = segmentwise_attributions[prediction_id] > np.percentile(segmentwise_attributions[prediction_id], 100-redaction1_percentile)
    new_im1=mask_on_image(image , mask, masking_type)
    return new_im1,mask
    

In [ ]:
#generate mask using list of all mask ids
def get_mask(mask_id_intersection,new_segs):
    
    numbered_masks = {ind: mask for ind, mask in enumerate(new_segs)}
    oldmask=np.zeros(numbered_masks[0].shape, dtype=bool)
    for i in mask_id_intersection:
        mask=np.logical_or(numbered_masks[i],oldmask)
        # plt.imshow(mask)
        # plt.show()
        oldmask=mask
    return mask

In [3]:
#generate mask using list of all mask ids
def get_mask_based_on_disjint_segs(new_segs,ranked_pred_list):
    numbered_masks = {ind: mask for ind, mask in enumerate(new_segs)}
    oldmask=np.zeros(numbered_masks[0].shape, dtype=bool)
    for i in ranked_pred_list:
        if i['required_for_pred']==True:
            
            mask=np.logical_or(numbered_masks[i['mask_id']],oldmask)
            # plt.imshow(mask)
            # plt.show()
            oldmask=mask
    return mask

In [8]:
#generate mask using list of all mask ids
def get_mask_based_on_disjint_segs_for_red_perc(new_segs,ranked_pred_list,redaction_percentile):
    numbered_masks = {ind: mask for ind, mask in enumerate(new_segs)}
    oldmask=np.zeros(numbered_masks[0].shape, dtype=bool)
    list_of_segs=[]
    redact_segments_for_this=[True,False]
    k=0
    for x in redact_segments_for_this:
        for i in ranked_pred_list:
            if i['required_for_pred']==x:
                list_of_segs.append(i['mask_id'])
                mask=np.logical_or(numbered_masks[i['mask_id']],oldmask)
                oldmask=mask
                if k+1==redaction_percentile:
                    return mask,list_of_segs
                k=k+1
    
    return mask,list_of_segs